# Interactive basis plaything

In [1]:
import numpy as np
import bqplot
from bqplot import pyplot as plt
from IPython.display import display
from ipywidgets import Layout
import numpy as np
import pandas as pd
%matplotlib notebook

from madminer.madminer import MadMiner

## Preparation

In [2]:
n_resolution = 50

In [3]:
miner = MadMiner()
miner.add_parameter(
    lha_block='dim6',
    lha_id=1,
    parameter_name='fW',
    morphing_max_power=4,
    morphing_parameter_range=(-1.,1.)
)
miner.add_parameter(
    lha_block='dim6',
    lha_id=2,
    parameter_name='fWW',
    morphing_max_power=4,
    morphing_parameter_range=(-1.,1.)
)

xi = np.linspace(-1.,1.,n_resolution)
yi = np.linspace(-1.,1.,n_resolution)
xx, yy = np.meshgrid(xi, yi)
xx = xx.reshape((-1,1))
yy = yy.reshape((-1,1))
theta_evaluation = np.hstack([xx, yy])

x_updated = False
y_updated = False

## Basis evaluation

In [4]:
def evaluate_basis(basis=None):
    
    # Set basis
    miner.set_benchmarks()
    if basis is not None:
        for theta in basis:
            miner.add_benchmark(
                {'fW':theta[0], 'fWW':theta[1]}
            )
            
    # Set up morphing
    miner.set_benchmarks_from_morphing(keep_existing_benchmarks=True,
                           n_trials=1)

    # Read out basis (which might have been appended)
    actual_basis = []
    for _, parameters in miner.benchmarks.items():
        actual_basis.append([value for _, value in parameters.items()])
    actual_basis = np.array(actual_basis)

    # Evaluate basis
    squared_weights = []

    for theta in theta_evaluation:
        wi = miner.current_morpher._calculate_morphing_weights(theta, None)
        squared_weights.append(np.sum(wi*wi)**0.5)

    squared_weights = np.array(squared_weights).reshape((n_resolution,n_resolution))
    
    return actual_basis, squared_weights

## Initial set up

In [5]:
basis = np.array([[ 0.        ,  0.        ],
       [-0.8318245 ,  0.85645093],
       [-0.82002127, -0.85191237],
       [ 0.76870769, -0.81272456],
       [ 0.7819962 ,  0.86242685],
       [-0.57243257,  0.37755934],
       [-0.29730939,  0.74563426],
       [ 0.13777926,  0.35254704],
       [ 0.46330191,  0.51783982],
       [ 0.64649576, -0.01232633],
       [ 0.16629182, -0.29365045],
       [ 0.39752054, -0.64235507],
       [-0.19238158, -0.59962178],
       [-0.30730345, -0.09697784],
       [-0.70631846, -0.18913046]])

basis, squared_weights = evaluate_basis(basis)

## Interactive tool

In [6]:
def update(change):
    global basis, squared_weights, x_updated, y_updated
    
    variable = change['name']
    values = change['new']
    
    if variable == 'x':
        basis[:,0] = values
        x_updated = True
    elif variable == 'y':
        basis[:,1] = values
        y_updated = True
    
    if x_updated and y_updated:
        basis, squared_weights = evaluate_basis(basis)
        heat.color = np.log(squared_weights) / np.log(10)
        
        x_updated = False
        y_updated = False

In [8]:
x_sc = bqplot.scales.LinearScale(min=-1., max=1.)
y_sc = bqplot.scales.LinearScale(min=-1., max=1.)
c_sc = bqplot.scales.ColorScale(min=0., mid=1., max=2., scheme='YlOrRd')

heat = bqplot.GridHeatMap(color=np.log(squared_weights) / np.log(10),
                          scales={'row': x_sc, 'column': y_sc, 'color': c_sc},
                          #stroke='white',
                          row=yi,
                          column=xi,
                          stroke=None
                         )

scatter = bqplot.Scatter(colors=['black'],
                         x=basis[:,0],
                         y=basis[:,1],
                         scales={'x': x_sc, 'y': y_sc})

ax_x = bqplot.Axis(scale=x_sc, label='fW')
ax_y = bqplot.Axis(scale=y_sc, orientation='vertical', label='fWW')
ax_c = bqplot.ColorAxis(scale=c_sc,
                        orientation='vertical', 
                        side='right')

fig = bqplot.Figure(marks=[heat, scatter],
                    axes=[ax_x, ax_y, ax_c],
                    layout=Layout(width='600px', height='600px'))

display(fig)

scatter.observe(update, ['y','x'])
scatter.enable_move = True


Figure(axes=[Axis(label='fW', scale=LinearScale(max=1.0, min=-1.0)), Axis(label='fWW', orientation='vertical', scale=LinearScale(max=1.0, min=-1.0)), ColorAxis(orientation='vertical', scale=ColorScale(max=2.0, mid=1.0, min=0.0, scheme='YlOrRd'), side='right')], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(height='600px', width='600px'), marks=[GridHeatMap(anchor_style={'fill': 'white', 'stroke': 'blue'}, color=array([[2.32657735, 2.28160512, 2.24608306, ..., 2.36927034, 2.46051482,
        2.54918499],
       [2.21201676, 2.14679153, 2.09170744, ..., 2.30453903, 2.41006681,
        2.50940785],
       [2.09174081, 1.99529323, 1.90585276, ..., 2.24536985, 2.36554086,
        2.47517553],
       ...,
       [2.75821   , 2.68489933, 2.60954857, ..., 1.77116069, 1.8525234 ,
        1.94854608],
       [2.78549059, 2.71695397, 2.64742883, ..., 2.00289022, 2.0459879 ,
        2.10372075],
       [2.81572664, 2.7521741 , 2.68857861, ..., 2.18123939, 2.20586043,
        2.24214734]]), column=array([-1.        , -0.95918367, -0.91836735, -0.87755102, -0.83673469,
       -0.79591837, -0.75510204, -0.71428571, -0.67346939, -0.63265306,
       -0.59183673, -0.55102041, -0.51020408, -0.46938776, -0.42857143,
       -0.3877551 , -0.34693878, -0.30612245, -0.26530612, -0.2244898 ,
       -0.18367347, -0.14285714, -0.10204082, -0.06122449, -0.02040816,
        0.02040816,  0.06122449,  0.10204082,  0.14285714,  0.18367347,
        0.2244898 ,  0.26530612,  0.30612245,  0.34693878,  0.3877551 ,
        0.42857143,  0.46938776,  0.51020408,  0.55102041,  0.59183673,
        0.63265306,  0.67346939,  0.71428571,  0.75510204,  0.79591837,
        0.83673469,  0.87755102,  0.91836735,  0.95918367,  1.        ]), interactions={'hover': 'tooltip'}, row=array([-1.        , -0.95918367, -0.91836735, -0.87755102, -0.83673469,
       -0.79591837, -0.75510204, -0.71428571, -0.67346939, -0.63265306,
       -0.59183673, -0.55102041, -0.51020408, -0.46938776, -0.42857143,
       -0.3877551 , -0.34693878, -0.30612245, -0.26530612, -0.2244898 ,
       -0.18367347, -0.14285714, -0.10204082, -0.06122449, -0.02040816,
        0.02040816,  0.06122449,  0.10204082,  0.14285714,  0.18367347,
        0.2244898 ,  0.26530612,  0.30612245,  0.34693878,  0.3877551 ,
        0.42857143,  0.46938776,  0.51020408,  0.55102041,  0.59183673,
        0.63265306,  0.67346939,  0.71428571,  0.75510204,  0.79591837,
        0.83673469,  0.87755102,  0.91836735,  0.95918367,  1.        ]), scales={'row': LinearScale(max=1.0, min=-1.0), 'column': LinearScale(max=1.0, min=-1.0), 'color': ColorScale(max=2.0, mid=1.0, min=0.0, scheme='YlOrRd')}, scales_metadata={'row': {'orientation': 'vertical', 'dimension': 'y'}, 'column': {'orientation': 'horizontal', 'dimension': 'x'}, 'color': {'dimension': 'color'}}, stroke=None, tooltip_style={'opacity': 0.9}), Scatter(colors=['black'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=1.0, min=-1.0), 'y': LinearScale(max=1.0, min=-1.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, x=array([-0.25325798, -0.62772606, -0.82002127,  0.76870769,  0.7819962 ,
       -0.50857713, -0.29730939,  0.21482713,  0.38078457,  0.79780585,
        0.16629182,  0.58503989, -0.10857713, -0.2362367 , -0.70631846]), y=array([ 0.26828756,  0.56873599, -0.85191237, -0.81272456,  0.86242685,
        0.1247898 ,  0.74563426,  0.6673907 ,  0.15169563,  0.28174047,
       -0.29365045, -0.10839406, -0.695838  , -0.21153307, -0.18913046]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0))